In [1]:
import numpy as np
import pandas as pd
import qml
import scipy.spatial as scs
import glob
import scipy.interpolate as sci
import functools
import matplotlib.pyplot as plt
%load_ext line_profiler
c = qml.Compound('../../test/c20.xyz')

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed',)).History will not be written to the database.


In [70]:
def rotate_2(refsite1, refsite2, ontosite1, ontosite2, coords = c.coordinates):
    coordinates = np.copy(coords)
    if refsite1 == ontosite1 and refsite2 == ontosite2:
        return coordinates
    
    # distances are similar and large enough?
    d1 = np.linalg.norm(coordinates[refsite1] - coordinates[refsite2])
    d2 = np.linalg.norm(coordinates[ontosite1] - coordinates[ontosite2])
    assert (d1 > 0.1)
    assert (d2 > 0.1)
    assert (abs(d1-d2)< 1e-2)
    
    # rotate site 1
    if refsite1 != ontosite1:
        a, b = coordinates[refsite1], coordinates[ontosite1]
        cross = np.cross(a, b)
        #print (cross)
        cross /= np.linalg.norm(cross)

        angle = np.arccos(np.dot(a, b) / (np.linalg.norm(a)*np.linalg.norm(b)))
        cross *= angle
        rot = scs.transform.Rotation.from_rotvec(cross)

        transformed = rot.apply(coordinates)
    else:
        transformed = coordinates.copy()
    
    # rotation matches site 1
    assert (np.linalg.norm(transformed[refsite1] - coordinates[ontosite1]) < 1e-4)
    # center of geometry has not moved
    assert (np.linalg.norm(coordinates.sum(axis=0) - transformed.sum(axis=0)) < 1e-4)
    
    # rotate site 2
    if (np.linalg.norm(coordinates[ontosite2] - transformed[refsite2]) < 1e-4):
        # coincidently already rotated in place
        transformed2 = transformed
    else:
        cross = coordinates[ontosite1]
        cross /= np.linalg.norm(cross)

        a = transformed[refsite2]
        aprime = a - np.dot(a, cross)*cross
        b = coordinates[ontosite2]
        bprime = b - np.dot(b, cross)*cross

        # left or right rotation
        if np.dot(np.cross(aprime, bprime), cross) > 0:
            sign = 1
        else:
            sign = -1

        # machine precision can bring |cos| > 1.
        cosangle = np.dot(aprime, bprime) / (np.linalg.norm(aprime)*np.linalg.norm(bprime))
        cosangle = np.sign(cosangle) * min(1., abs(cosangle))

        angle = sign*np.arccos(cosangle)
        #print (np.degrees(angle), sign, np.dot(aprime, bprime) / (np.linalg.norm(aprime)*np.linalg.norm(bprime)))
        cross *= angle
        rot = scs.transform.Rotation.from_rotvec(cross)

        # mapping vectors are nonzero
        assert (np.linalg.norm(aprime) > 0.1)
        assert (np.linalg.norm(bprime) > 0.1)

        transformed2 = rot.apply(transformed)
    
    # site 1 has not changed
    assert (np.linalg.norm(transformed[refsite1] - transformed2[refsite1]) < 1e-4)
    
    # rotation matches site 2
    #print (np.linalg.norm(coordinates[ontosite2] - transformed2[refsite2]))
    assert (np.linalg.norm(coordinates[ontosite2] - transformed2[refsite2]) < 1e-4)
    
    return coordinates
rotate_2(0, 16, 1, 17)

array([[-7.47063080e-01, -1.02824412e+00,  1.66373333e+00],
       [-1.20877346e+00,  3.92754310e-01,  1.66373333e+00],
       [-0.00000000e+00,  1.27097963e+00,  1.66373333e+00],
       [ 1.20877346e+00,  3.92754310e-01,  1.66373333e+00],
       [ 7.47063080e-01, -1.02824412e+00,  1.66373333e+00],
       [-1.20877187e+00, -1.66373428e+00,  3.92755160e-01],
       [-1.95583660e+00, -6.35488440e-01, -3.92753720e-01],
       [-1.95583588e+00,  6.35490290e-01,  3.92754300e-01],
       [-1.20877295e+00,  1.66373389e+00, -3.92753530e-01],
       [-1.30000000e-07,  2.05648767e+00,  3.92754760e-01],
       [ 1.20877402e+00,  1.66373326e+00, -3.92752910e-01],
       [ 1.95583604e+00,  6.35489520e-01,  3.92754760e-01],
       [ 1.95583634e+00, -6.35489350e-01, -3.92753530e-01],
       [ 1.20877248e+00, -1.66373345e+00,  3.92756830e-01],
       [ 9.80000000e-07, -2.05648829e+00, -3.92751510e-01],
       [-7.47061760e-01,  1.02824567e+00, -1.66373297e+00],
       [ 7.47063080e-01,  1.02824565e+00

In [71]:
def test_rotate_2():
    for i in range(20):
        for j in range(20):
            if i == j:
                continue
            for k in range(20):
                for l in range(20):
                    if k == l:
                        continue
                    d1 = np.linalg.norm(c.coordinates[i] - c.coordinates[j])
                    d2 = np.linalg.norm(c.coordinates[k] - c.coordinates[l])
                    if abs(d1-d2) > 1e-2:
                        continue
                    try:
                        rotate_2(i, j, k, l)
                    except:
                        print (i, j, k, l)
                        raise
test_rotate_2()

In [48]:
np.sign(10)

1